In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

TRRNA_THRESHOLD=300
NUM_CPU=os.cpu_count()
SEED=42

In [2]:
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/protenix-0.4.6-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/rdkit-2024.9.6-cp310-cp310-manylinux_2_28_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/ml_collections-1.1.0-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/biopython-1.85-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/pyrosetta-2025.13-cp310-cp310-linux_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/blosc-1.11.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/ml_collections-1.1.0-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/biotraj-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/biotite-1.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/biopandas-0.5.1-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/looseversion-1.1.2-py3-none-any.whl'

Processing /kaggle/input/dependencies-tr-pr/protenix-0.4.6-py3-none-any.whl
Processing /kaggle/input/dependencies-tr-pr/rdkit-2024.9.6-cp310-cp310-manylinux_2_28_x86_64.whl
Processing /kaggle/input/dependencies-tr-pr/ml_collections-1.1.0-py3-none-any.whl
Processing /kaggle/input/dependencies-tr-pr/biopython-1.85-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/dependencies-tr-pr/pyrosetta-2025.13-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/dependencies-tr-pr/blosc-1.11.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/dependencies-tr-pr/ml_collections-1.1.0-py3-none-any.whl
ml-collections is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/dependencies-tr-pr/biotraj-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/dependencies-tr-pr/biotite-1.0.1-cp310-cp310-manylinux

In [3]:
import pandas as pd
test_df=pd.read_csv('/kaggle/input/stanford-rna-3d-folding/test_sequences.csv')

In [4]:
trRNA_input_ids = []
results = []
for i, seq in enumerate(test_df.sequence):
    target_id=test_df.target_id[i]
    if len(seq)<=TRRNA_THRESHOLD:
        trRNA_input_ids.append(target_id)
        print('Send to trRNA', target_id)
    else:
        print(f"Set all 0: {target_id}")
        result=pd.DataFrame(columns=['ID', 'resname', 'resid', 
                                     'x_1', 'y_1', 'z_1', 
                                     'x_2', 'y_2', 'z_2',
                                     'x_3', 'y_3', 'z_3', 
                                     'x_4', 'y_4', 'z_4', 
                                     'x_5', 'y_5', 'z_5'], 
                                     data=[[target_id, x, j+1] + [0.0]*15 for j, x in enumerate(seq)])
        result['ID']=result.apply(lambda x: x.ID + '_' + str(x.resid), axis=1)
        results.append(result)
        result.to_csv('submission.csv', index=False, mode='a', header=(i==0))

Send to trRNA R1107
Send to trRNA R1108
Send to trRNA R1116
Send to trRNA R1117v2
Set all 0: R1126
Send to trRNA R1128
Set all 0: R1136
Set all 0: R1138
Send to trRNA R1149
Send to trRNA R1156
Send to trRNA R1189
Send to trRNA R1190


In [5]:
import pdb
import os
import tempfile
import shutil
import subprocess

import tempfile

import glob

import string

import json
import os
import sys
import numpy as np
import torch
import torch.nn as nn

from collections import defaultdict
from argparse import ArgumentParser
from pathlib import Path

In [6]:
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

In [7]:
torch.cuda.is_available(),torch.cuda.device_count()

(True, 1)

In [8]:
import os
import sys
import shutil
from pathlib import Path

input_base = '/kaggle/input'
work_base = '/kaggle/working'

required_dirs = [
    'folding',
    'network',
    'spot-rna',
    'model-1'
]

shutil.rmtree(work_base, ignore_errors=True)  # 慎用！确保你知道自己在做什么
os.makedirs(work_base, exist_ok=True)

for dir_name in required_dirs:
    src = os.path.join(input_base, dir_name)
    dst = os.path.join(work_base, dir_name)
    
    if os.path.exists(src):
        shutil.copytree(src, dst)
        print(f"Copied: {src} => {dst}")
    else:
        raise FileNotFoundError(f"关键目录缺失: {src}")
    
sys.path.append("/kaggle/working/")

Copied: /kaggle/input/folding => /kaggle/working/folding
Copied: /kaggle/input/network => /kaggle/working/network
Copied: /kaggle/input/spot-rna => /kaggle/working/spot-rna
Copied: /kaggle/input/model-1 => /kaggle/working/model-1


In [9]:
torch.cuda.is_available(),torch.cuda.device_count()

(True, 1)

In [10]:
import os
import numpy as np
import string


def parse_a3m(filename, limit=20000, rm_query_gap=True):
    seqs = []
    table = str.maketrans(dict.fromkeys(string.ascii_lowercase))

    # read file line by line
    n = 0
    for line in open(filename, "r"):
        if line[0] != '>' and len(line.strip()) > 0:
            seqs.append(
                line.rstrip().replace('W', 'A').replace('R', 'A').replace('Y', 'C').replace('E', 'A').replace('I',
                                                                                                              'A').replace(
                    'P', 'G').replace('T', 'U').translate(table))
            n += 1
            if n == limit:
                break

    # convert letters into numbers
    alphabet = np.array(list("AUCG-"), dtype='|S1').view(np.uint8)
    msa = np.array([list(s) for s in seqs], dtype='|S1').view(np.uint8)
    for i in range(alphabet.shape[0]):
        msa[msa == alphabet[i]] = i

        # treat all unknown characters as gaps
    msa[msa > 4] = 4
    if rm_query_gap:
        return msa[:, msa[0] < 4]
    return msa


def ss2mat(ss_seq):
    ss_mat = np.zeros((len(ss_seq), len(ss_seq)))
    stack = []
    stack1 = []
    stack2 = []
    stack3 = []
    stack_alpha = {alpha: [] for alpha in string.ascii_lowercase}
    for i, s in enumerate(ss_seq):
        if s == '(':
            stack.append(i)
        elif s == ')':
            ss_mat[i, stack.pop()] = 1
        elif s == '[':
            stack1.append(i)
        elif s == ']':
            ss_mat[i, stack1.pop()] = 1
        elif s == '{':
            stack2.append(i)
        elif s == '}':
            ss_mat[i, stack2.pop()] = 1
        elif s == '<':
            stack3.append(i)
        elif s == '>':
            ss_mat[i, stack3.pop()] = 1
        elif s.isalpha() and s.isupper():
            stack_alpha[s.lower()].append(i)
        elif s.isalpha() and s.islower():
            ss_mat[i, stack_alpha[s].pop()] = 1
        elif s in ['.', ',', '_', ':', '-']:
            continue
        else:
            raise ValueError(f'unk not: {s}!')
    allstacks = stack + stack1 + stack2 + stack3
    for _, stack in stack_alpha.items():
        allstacks += stack
    if len(allstacks) > 0:
        raise ValueError('Provided dot-bracket notation is not completely matched!')

    ss_mat += ss_mat.T
    return ss_mat


def parse_ct(ct_file, length=None):
    seq_ct = ''
    if length is None:
        length = int(open(ct_file).readlines()[0].split()[0])
    mat = np.zeros((length, length))
    for line in open(ct_file):
        items = line.split()
        if len(items) >= 6 and items[0].isnumeric() and items[2].isnumeric() and items[3].isnumeric() and items[
            4].isnumeric():
            seq_ct += items[1]
            if int(items[4]) > 0:
                mat[int(items[4]) - 1, int(items[5]) - 1] = 1
                mat[int(items[5]) - 1, int(items[4]) - 1] = 1
    return mat

In [11]:
pkg_dir = '/kaggle/working'
from network.RNAformer import DistPredictor
from network.config import n_bins, obj
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [12]:
import time
from datetime import datetime

def predict(model, msa, ss_, args, window=150, shift=50):
    start_time = time.time()
    
    if ss_.shape[0] != msa.shape[-1]:
        raise ValueError(f'ss length {ss_.shape[0]}, msa length {msa.shape[1]}!')
    with torch.no_grad():
        feat = torch.from_numpy(msa).to(device)
        ss_ = torch.from_numpy(ss_).to(device)
        L = msa.shape[-1]
        res_id = torch.arange(L, device=device).view(1, L)
        if L > 300:  # predict by crops for long RNA
            pred_dict = {
                'contact': torch.zeros((L, L), device=device),
                'distance': {k: torch.zeros((L, L, n_bins['2D']['distance']), device=device) for k in
                             obj['2D']['distance']},
            }

            count_1d = torch.zeros((L)).to(device)
            count_2d = torch.zeros((L, L)).to(device)
            
            grids = np.arange(0, L - window + shift, shift)
            ngrids = grids.shape[0]
            print("ngrid:     ", ngrids)
            print("grids:     ", grids)
            print("windows:   ", window)

            idx_pdb = torch.arange(L).long().view(1, L)
            for i in range(ngrids):
                for j in range(i, ngrids):
                    start_1 = grids[i]
                    end_1 = min(grids[i] + window, L)
                    start_2 = grids[j]
                    end_2 = min(grids[j] + window, L)
                    sel = np.zeros((L)).astype(np.bool_)
                    sel[start_1:end_1] = True
                    sel[start_2:end_2] = True

                    input_msa = feat[:, sel]
                    input_ss = ss_[sel][:, sel]
                    mask = torch.sum(input_msa == 4, dim=-1) < .7 * sel.sum()  # remove too gappy sequences

                    input_msa = input_msa[mask]
                    input_idx = idx_pdb[:, sel]
                    input_res_id = res_id[:, sel]

                    print("running crop: %d-%d/%d-%d" % (start_1, end_1, start_2, end_2), input_msa.shape)
                    pred_gemos = model(input_msa, input_ss, res_id=input_res_id.to(device),
                                       msa_cutoff=args.nrows)['geoms']
                    weight = 1
                    sub_idx = input_idx[0].cpu()
                    sub_idx_2d = np.ix_(sub_idx, sub_idx)
                    count_2d[sub_idx_2d] += weight
                    count_1d[sub_idx] += weight

                    for k in obj['2D']:
                        if k == 'contact':
                            pred_dict['contact'][sub_idx_2d] += weight * pred_gemos['contact']
                        else:
                            for a in obj['2D'][k]:
                                pred_dict[k][a][sub_idx_2d] += weight * pred_gemos[k][a]
                    
                    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    print(f"[{current_time}] Completed crop {i}-{j}, took {time.time() - start_time:.2f} seconds")
                    
            for k in obj['2D']:
                if k == 'contact':
                    pred_dict['contact'] /= count_2d
                else:
                    for a in obj['2D'][k]:
                        if pred_dict[k][a].size().__len__() == 3:
                            pred_dict[k][a] /= count_2d[:, :, None]
                        else:
                            pred_dict[k][a] /= count_2d
        else:
            pred_dict = model(feat, ss_, res_id=res_id.to(device), msa_cutoff=args.nrows)['geoms']
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"[{current_time}] Completed prediction, took {time.time() - start_time:.2f} seconds")

    for l in pred_dict:
        if isinstance(pred_dict[l], dict):
            for k in pred_dict[l]:
                pred_dict[l][k] = pred_dict[l][k].cpu().detach().numpy()
        else:
            pred_dict[l] = pred_dict[l].cpu().detach().numpy()

    return pred_dict


In [13]:
def main(args):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu)
    torch.set_num_threads(args.cpu)
    device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    py = sys.executable

    out_dir = os.path.dirname(os.path.abspath(args.npz))
    os.makedirs(out_dir, exist_ok=True)

    cwd = os.getcwd()
    msa = parse_a3m(args.msa, limit=20000)

    # Check if ss_file is not provided
    if args.ss_file is None:
        # Predict SS by SPOT-RNA
        print('predict SS by SPOT-RNA')

        # Create a temporary directory
        tmp_dir = tempfile.TemporaryDirectory(prefix=out_dir + '/')
        spot_out_dir = tmp_dir.name

        # Check if the seq.fasta file doesn't exist, and create it by extracting the first 2 lines from the MSA file
        if not os.path.isfile(f'{spot_out_dir}/seq.fasta'):
            with open(f'{spot_out_dir}/seq.fasta', 'w') as fasta_file:
                with open(args.msa, 'r') as msa_file:
                    # Write the first 2 lines of MSA file to seq.fasta
                    for _ in range(2):
                        line = msa_file.readline()
                        fasta_file.write(line)

        # Modify Python environment path for SPOT-RNA
        # spot_py = py.replace('trRNA', 'spot_venv')
        spot_py = py

        # Change current working directory to SPOT_RNA directory
        os.chdir(f'{pkg_dir}/spot-rna')

        # If 'utils' directory exists, rename it to 'utils_spot'
        if os.path.isdir(f'utils'):
            shutil.move(f'utils', f'utils_spot')

        # Modify SPOT-RNA.py to use the new utils_spot directory
        with open('SPOT-RNA.py', 'r') as spot_script_file:
            spot_script = spot_script_file.read()


        # Run SPOT-RNA using subprocess with nohup equivalent in Python
        # print(spot_py)
        # print(spot_out_dir)
        with open(f'{out_dir}/spot.log', 'w') as log_file:
            # pdb.set_trace()
            subprocess.run(
                [spot_py, 'SPOT-RNA.py', '--inputs', f'{spot_out_dir}/seq.fasta', '--outputs', spot_out_dir, '--gpu', str(args.gpu)],
                stdout=log_file, stderr=subprocess.STDOUT
            )

        # Return to the original working directory
        os.chdir(cwd)

        prob_files = glob.glob(f'{spot_out_dir}/*.prob')
        if len(prob_files) == 0: raise ValueError(
            f'Fails to predict SS! Please refer to {out_dir}/spot.log to see what happened.')
        ss = np.loadtxt(prob_files[0])
        if (np.tril(ss) == 0).all() or (np.triu(ss) == 0).all():
            ss += ss.T
    else:
        if args.ss_fmt == 'dot_bracket':
            ss = ss2mat(open(args.ss_file).read().rstrip().splitlines()[-1].strip())
        elif args.ss_fmt == 'ct':
            ss = parse_ct(args.ss_file, length=len(msa[0]))
        elif args.ss_fmt == 'spot_prob':
            ss = np.loadtxt(args.ss_file)
            ss += ss.T
        if len(ss) != len(msa[0]):
            raise ValueError(f'The SS shape {ss.shape} mismatches the MSA shape {msa.shape}!')

    print('predict geometries')
    config = json.load(open(f'{args.model_pth}/config/model_1.json', 'r'))

    model = DistPredictor(dim_2d=config['channels'], layers_2d=config['n_blocks'])

    model_ckpt = torch.load(f'{args.model_pth}/models/model_1.pth.tar', map_location=device)
    model.load_state_dict(model_ckpt)
    model.eval()
    model.to(device)

    pred = predict(model, msa, ss, args=args)

    print('done!')
    print('saving......')
    np.savez_compressed(args.npz, **pred)


In [14]:
from folding.utils_cst import npz2cst
from folding.utils_ros import fold_from_cst

In [15]:
def fold(args):
    os.makedirs(os.path.dirname(os.path.abspath(args.OUT)), exist_ok=True)

    tmpdir = tempfile.TemporaryDirectory(prefix=args.TMPDIR + '/')
    args.tmpdir = tmpdir.name
    print('temp folder:     ', tmpdir.name)

    # parse npz into rosetta-format restraint files
    npz2cst(args)

    original_out = args.OUT
    base, ext = os.path.splitext(original_out)
    
    # perform energy minimization for each model
    args.OUT = f"{base}_model_1{ext}"
    fold_from_cst(args)
    for i in range(2,args.nmodels+1):
        os.system(f"cp {tmpdir.name}/model_{i}.pdb {base}_model_{i}{ext}");
    
    args.OUT = original_out

In [16]:
import os
import pandas as pd
from biopandas.pdb import PandasPdb

def get_c1_coords(base="/kaggle/working/output_model_model", ext=".pdb",nmodels=5):
    c1_coords_all_models = {}
    for i in range(1, nmodels+1):
        model_path = f"{base}_{i}{ext}"
        ppdb = PandasPdb().read_pdb(model_path)
        atom_df = ppdb.df['ATOM']
        
        for resid, group in atom_df.groupby('residue_number'):
            resname = group['residue_name'].iloc[0]
            c1_coords = group[group['atom_name'] == "C1'"][['x_coord', 'y_coord', 'z_coord']].values
            
            if resid not in c1_coords_all_models:
                c1_coords_all_models[resid] = {'resname': resname, 'coords': []}
                
            if len(c1_coords) > 0:
                c1_coords_all_models[resid]['coords'].extend(c1_coords[0])
            else:
                c1_coords_all_models[resid]['coords'].extend([0.0, 0.0, 0.0])

    rna_id = os.path.basename(base).split('_')[1]
        
    result=pd.DataFrame(columns=['ID', 'resname', 'resid'] + 
    [item for sublist in [[f'x_{j+1}', f'y_{j+1}', f'z_{j+1}'] for j in range(int(len(c1_coords_all_models[1]['coords'])/3))] for item in sublist], 
                        data=[[f"{rna_id}_{resid}", data['resname'], resid] + 
                              [data['coords'][j] for j in range(len(data['coords']))] for resid, data in c1_coords_all_models.items()])
    return result

In [17]:
ori_dir='/kaggle/input/stanford-rna-3d-folding/MSA'
target_dir='/kaggle/working/simplified-msa/MSA1'
os.makedirs(target_dir, exist_ok=True)
for filename in os.listdir(ori_dir):
    basename=filename.split('.')[0]
    if(basename not in trRNA_input_ids):
        continue
    if filename.endswith('.fasta'):
        with open(os.path.join(ori_dir, filename), 'r') as f:
            lines = f.readlines()
            with open(os.path.join(target_dir, filename), 'w') as wf:
                wf.writelines(lines[:2])

In [18]:
class Main_Args:
    msa="/kaggle/input/simplified-msa/MSA1/R1107.MSA.fasta"
    npz = "/kaggle/working/R1107.npz"
    ss_file = None
    ss_fmt = "dot_bracket"
    gpu = "0"
    cpu = NUM_CPU
    model_pth="/kaggle/working/model-1"
    nrows=1000

In [19]:
class Fold_Args:
    NPZ = "/kaggle/working/R1107.npz"
    FASTA = "/kaggle/input/simplified-msa/MSA1/R1107.MSA.fasta"
    OUT = "/kaggle/working/output_model.pdb"
    
    TMPDIR = "/kaggle/working"
    nmodels = 5
    dcut = 0.45
    CPU = NUM_CPU

In [20]:
main_args = Main_Args()
fold_args = Fold_Args()

for i,input_id in enumerate(trRNA_input_ids):
    main_args.npz = f'/kaggle/working/{input_id}.npz'
    main_args.msa = f'/kaggle/working/simplified-msa/MSA1/{input_id}.MSA.fasta'
    fold_args.NPZ = f'/kaggle/working/{input_id}.npz'
    fold_args.FASTA = f'/kaggle/working/simplified-msa/MSA1/{input_id}.MSA.fasta'
    fold_args.OUT = f'/kaggle/working/output_{input_id}.pdb'
    print('main_args',main_args)
    print('fold_args',fold_args)
    main(main_args)
    fold(fold_args)
    result=get_c1_coords(base=f'/kaggle/working/output_{input_id}_model', ext='.pdb',nmodels=fold_args.nmodels)
        
        
    # result.to_csv('submission.csv', index=False, mode='a', header=(i==0 and len(results)==0))
    results.append(result)
    torch.cuda.empty_cache()
    all_results = pd.concat(results, ignore_index=True)
    all_results.to_csv('submission.csv', index=False, header=True)

main_args <__main__.Main_Args object at 0x7a9f354a4ca0>
fold_args <__main__.Fold_Args object at 0x7a9f354a4c70>
predict SS by SPOT-RNA
predict geometries
[2025-05-21 15:15:41] Completed prediction, took 2.38 seconds
done!
saving......
temp folder:      /kaggle/working/3x0ai4qq
┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python310.ubuntu.cxx11thread.

In [21]:
all_results = pd.concat(results, ignore_index=True)
all_results.to_csv('submission.csv', index=False, header=True)

In [22]:
display(pd.read_csv('submission.csv'))

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,R1126_1,G,1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,R1126_2,G,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,R1126_3,A,3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,R1126_4,A,4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,R1126_5,U,5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,R1190_114,U,114,7.228,2.539,-28.886,-12.515,9.116,-22.478,-12.466,2.775,-23.210,-3.847,10.206,-24.925,-2.866,12.242,-27.158
2511,R1190_115,U,115,6.235,5.619,-24.747,-7.226,8.825,-21.215,-8.830,5.627,-19.044,0.842,10.260,-22.142,-4.783,9.045,-22.631
2512,R1190_116,U,116,4.631,9.884,-22.086,-2.088,10.610,-21.313,-4.859,8.437,-19.215,6.640,11.886,-19.919,-7.201,6.945,-20.051
2513,R1190_117,U,117,1.557,14.039,-23.049,2.391,12.868,-23.026,-2.947,12.704,-19.656,9.911,13.537,-21.531,-11.271,2.997,-19.843
